**First Approch : Traditional Method**

In the first approach, I implemented and evaluated four different machine learning models:

**Logistic Regression**,

**Random Forest**,

**XGBoost Classifier,**

**and LightGBM Classifier**

 Each model was trained separately on the dataset to predict loan defaults. After training, the performance of each model was assessed by calculating its accuracy. This approach helps compare the effectiveness of various algorithms in solving the classification problem. The results provide insights into which model performs best for loan default prediction.

This is common for all four Models :    

This code loads and cleans the dataset by removing irrelevant columns and encoding categorical features. It then splits the data into features (X) and target (y), standardizes the numerical features, and prepares it for training and validation.

In [58]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Load the dataset
file_path = '/content/train_data.xlsx'  # Update path if needed
data = pd.read_excel(file_path)

# Display the first few rows of the dataset
print("Dataset Preview:")
print(data.head())

# Drop irrelevant columns
data_cleaned = data.drop(columns=['customer_id', 'transaction_date'])

# Encode categorical variables
categorical_cols = data_cleaned.select_dtypes(include=['object']).columns
label_encoders = {col: LabelEncoder() for col in categorical_cols}
for col in categorical_cols:
    data_cleaned[col] = label_encoders[col].fit_transform(data_cleaned[col])

# Separate features and target variable
X = data_cleaned.drop(columns=['loan_status'])
y = data_cleaned['loan_status']

# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Dataset Preview:
   customer_id transaction_date sub_grade        term home_ownership  \
0     10608026       2014-01-01        C5   36 months       MORTGAGE   
1     10235120       2014-01-01        E5   36 months       MORTGAGE   
2     10705805       2014-01-01        D2   36 months       MORTGAGE   
3     11044991       2014-01-01        B4   36 months       MORTGAGE   
4     10161054       2014-01-01        C3   60 months       MORTGAGE   

   cibil_score  total_no_of_acc  annual_inc  int_rate             purpose  \
0          665                9     70000.0     16.24  debt_consolidation   
1          660                8     65000.0     23.40    home_improvement   
2          660                7     73000.0     17.57               other   
3          690                5    118000.0     12.85  debt_consolidation   
4          665                5     63000.0     14.98  debt_consolidation   

   loan_amnt application_type  installment verification_status  account_bal  \
0       

**LogisticRegression**

This code applies **SMOTE** for handling class imbalance, tunes a Logistic Regression model with **GridSearchCV**, and evaluates its performance using accuracy, ROC AUC score, confusion matrix, and classification report.

Logistic Regression is important for its simplicity, interpretability, and efficiency in binary classification tasks like predicting loan defaults.

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

# Handle class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Initialize Logistic Regression with class weighting
logistic_model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')

# Hyperparameter tuning using GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_smote, y_train_smote)

# Best model
best_logistic_model = grid_search.best_estimator_

# Predict on validation set
y_pred_proba = best_logistic_model.predict_proba(X_val)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)

# Display evaluation metrics
print("\n================== Logistic Regression Evaluation ==================")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc:.2f}\n")

print("================== Confusion Matrix ==================")
print(pd.DataFrame(conf_matrix,
                   index=['Actual Non-Default (0)', 'Actual Default (1)'],
                   columns=['Predicted Non-Default (0)', 'Predicted Default (1)']))
print("\n")

print("================ Classification Report ================")
print(class_report)



================== Logistic Regression Evaluation ==================
Accuracy: 69.66%
ROC AUC Score: 0.73

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actual Non-Default (0)                       3715                   2202
Actual Default (1)                           4698                  12126


================ Classification Report ================
              precision    recall  f1-score   support

           0       0.44      0.63      0.52      5917
           1       0.85      0.72      0.78     16824

    accuracy                           0.70     22741
   macro avg       0.64      0.67      0.65     22741
weighted avg       0.74      0.70      0.71     22741



**RandomForestClassifier**

This code trains a Random Forest classifier with **class weighting** to handle imbalanced data, evaluates its performance using accuracy, ROC AUC score, confusion matrix, and classification report.

Random Forest is valuable for its ability to handle complex datasets, reduce overfitting, and provide feature importance insights, making it useful for classification problems like loan default prediction.

In [36]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest with class weighting
rf_model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=12, class_weight='balanced')

# Train the model
rf_model.fit(X_train_smote, y_train_smote)

# Predict on validation set
y_pred_rf = rf_model.predict(X_val)

# Evaluate the model
accuracy_rf = accuracy_score(y_val, y_pred_rf)
roc_auc_rf = roc_auc_score(y_val, rf_model.predict_proba(X_val)[:, 1])
conf_matrix_rf = confusion_matrix(y_val, y_pred_rf)
class_report_rf = classification_report(y_val, y_pred_rf)

# Display evaluation metrics
print("\n================== Random Forest Evaluation ==================")
print(f"Accuracy: {accuracy_rf * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc_rf:.2f}\n")

print("================== Confusion Matrix ==================")
print(pd.DataFrame(conf_matrix_rf,
                   index=['Actual Non-Default (0)', 'Actual Default (1)'],
                   columns=['Predicted Non-Default (0)', 'Predicted Default (1)']))
print("\n")

print("================ Classification Report ================")
print(class_report_rf)



================== Random Forest Evaluation ==================
Accuracy: 72.89%
ROC AUC Score: 0.74

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actual Non-Default (0)                       3202                   2715
Actual Default (1)                           3449                  13375


================ Classification Report ================
              precision    recall  f1-score   support

           0       0.48      0.54      0.51      5917
           1       0.83      0.79      0.81     16824

    accuracy                           0.73     22741
   macro avg       0.66      0.67      0.66     22741
weighted avg       0.74      0.73      0.73     22741



**XGBClassifier**

This code trains an XGBoost classifier with optimized **hyperparameters**, evaluates its performance using accuracy, ROC AUC score, confusion matrix, and classification report.

XGBoost is particularly useful for handling imbalanced datasets, improving model accuracy with its advanced boosting algorithm, and providing robust performance in predicting loan defaults.

In [37]:
from xgboost import XGBClassifier

# Initialize XGBoost with optimal parameters
xgb_model = XGBClassifier(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.05, scale_pos_weight=10)

# Train the model
xgb_model.fit(X_train_smote, y_train_smote)

# Predict on validation set
y_pred_xgb = xgb_model.predict(X_val)

# Evaluate the model
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_val, xgb_model.predict_proba(X_val)[:, 1])
conf_matrix_xgb = confusion_matrix(y_val, y_pred_xgb)
class_report_xgb = classification_report(y_val, y_pred_xgb)

# Display evaluation metrics
print("\n================== XGBoost Evaluation ==================")
print(f"Accuracy: {accuracy_xgb * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc_xgb:.2f}\n")

print("================== Confusion Matrix ==================")
print(pd.DataFrame(conf_matrix_xgb,
                   index=['Actual Non-Default (0)', 'Actual Default (1)'],
                   columns=['Predicted Non-Default (0)', 'Predicted Default (1)']))
print("\n")

print("================ Classification Report ================")
print(class_report_xgb)



================== XGBoost Evaluation ==================
Accuracy: 73.98%
ROC AUC Score: 0.74

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actual Non-Default (0)                          4                   5913
Actual Default (1)                              4                  16820


================ Classification Report ================
              precision    recall  f1-score   support

           0       0.50      0.00      0.00      5917
           1       0.74      1.00      0.85     16824

    accuracy                           0.74     22741
   macro avg       0.62      0.50      0.43     22741
weighted avg       0.68      0.74      0.63     22741



**LGBMClassifier**

This code trains a LightGBM classifier with **optimized hyperparameters**, evaluates its performance using accuracy, ROC AUC score, confusion matrix, and classification report.

LightGBM is efficient for large datasets and imbalanced classes, providing fast and accurate results in predicting loan defaults.

In [38]:
from lightgbm import LGBMClassifier

# Initialize LightGBM with optimal parameters
lgbm_model = LGBMClassifier(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.05, scale_pos_weight=10)

# Train the model
lgbm_model.fit(X_train_smote, y_train_smote)

# Predict on validation set
y_pred_lgbm = lgbm_model.predict(X_val)

# Evaluate the model
accuracy_lgbm = accuracy_score(y_val, y_pred_lgbm)
roc_auc_lgbm = roc_auc_score(y_val, lgbm_model.predict_proba(X_val)[:, 1])
conf_matrix_lgbm = confusion_matrix(y_val, y_pred_lgbm)
class_report_lgbm = classification_report(y_val, y_pred_lgbm)

# Display evaluation metrics
print("\n================== LightGBM Evaluation ==================")
print(f"Accuracy: {accuracy_lgbm * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc_lgbm:.2f}\n")

print("================== Confusion Matrix ==================")
print(pd.DataFrame(conf_matrix_lgbm,
                   index=['Actual Non-Default (0)', 'Actual Default (1)'],
                   columns=['Predicted Non-Default (0)', 'Predicted Default (1)']))
print("\n")

print("================ Classification Report ================")
print(class_report_lgbm)


[LightGBM] [Info] Number of positive: 67192, number of negative: 67192
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009321 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3067
[LightGBM] [Info] Number of data points in the train set: 134384, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

================== LightGBM Evaluation ==================
Accuracy: 73.98%
ROC AUC Score: 0.74

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actual Non-Default (0)                          0                   5917
Actual Default (1)                              0                  16824


================ Classification Report ================
              precision    recall  f1-score   support

           0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**First Approach Summary:**

In the first approach, we applied four popular machine learning models (Logistic Regression, Random Forest, XGBoost, and LightGBM) to predict loan defaults. We preprocessed the data by handling class imbalance using SMOTE, encoded categorical features, and standardized the numerical ones. After training each model, we evaluated their performance using accuracy, ROC AUC scores, confusion matrices, and classification reports.

Model Performance:

Logistic Regression: Achieved an accuracy of 69.66% with a ROC AUC score of 0.73.

Random Forest: Achieved an accuracy of 72.89% with a ROC AUC score of 0.74.

XGBoost: Achieved an accuracy of 73.98% with a ROC AUC score of 0.74.

LightGBM: Achieved an accuracy of 73.98% with a ROC AUC score of 0.74.

**Conclusion:**

While the models performed reasonably well, the accuracy and ROC AUC scores indicate that there is room for improvement. Based on these results, we observe that we haven't yet found the optimal model, and further optimization or tuning might be necessary to improve the predictive performance and robustness of the model. Thus, we should explore additional techniques such as hyperparameter tuning, ensemble methods, or even a combination of models to achieve better results.



---



**Second Approach: Ensemble Learning with Voting Classifier**


In the second approach, we move towards an ensemble learning method to improve the model's performance.

Instead of relying on a single algorithm, we combine the predictions from multiple models (Logistic Regression, Random Forest, XGBoost, and LightGBM) using a Voting Classifier. The Voting Classifier combines the strengths of different algorithms and makes the final prediction by majority voting.

This approach helps in capturing diverse patterns in the data, potentially improving accuracy and robustness.

**STEP 1:**

In [47]:
!pip install sklearn
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


**STEP 2:**

This code performs data preprocessing by loading the dataset, handling missing values, and creating derived features such as debt-to-income ratio and employment stability. It also applies feature engineering, encodes categorical variables, and prepares the dataset for model training using a train-test split.

In [48]:
# Load Dataset
file_path = '/content/train_data.xlsx'
data = pd.read_excel(file_path)

# Convert 'term' column to numeric, handling errors
data['term'] = pd.to_numeric(data['term'], errors='coerce')

# Feature Engineering: Create Derived Features
data['debt_to_income'] = data['loan_amnt'] / (data['annual_inc'] + 1)
data['employment_stability'] = data['emp_length'] / (data['term'] + 1)

# Drop Low-Relevance Columns
data_cleaned = data.drop(columns=['customer_id', 'transaction_date'])

# Encode Categorical Variables with Imputation
categorical_cols = data_cleaned.select_dtypes(include=['object']).columns
numerical_cols = ['loan_amnt', 'annual_inc', 'cibil_score', 'account_bal', 'debt_to_income', 'employment_stability']

# Create a pipeline with imputation for numerical features
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Use mean imputation
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


# Separate Features and Target Variable
X = data_cleaned.drop(columns=['loan_status'])
y = data_cleaned['loan_status']

# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [49]:
# SMOTE for Balancing Classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(preprocessor.fit_transform(X_train), y_train)
X_val_transformed = preprocessor.transform(X_val)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['employment_stability']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['employment_stability']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


**STEP 3:**

This code trains four machine learning models—Logistic Regression, Random Forest, XGBoost, and LightGBM—on the balanced training dataset. Each model is initialized with specific hyperparameters and fitted to the training data to predict loan defaults.

In [50]:
# Logistic Regression
logistic_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
logistic_model.fit(X_train_balanced, y_train_balanced)

# Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=12, class_weight='balanced')
rf_model.fit(X_train_balanced, y_train_balanced)

# XGBoost
xgb_model = XGBClassifier(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.05, scale_pos_weight=10)
xgb_model.fit(X_train_balanced, y_train_balanced)

# LightGBM
lgbm_model = LGBMClassifier(random_state=42, n_estimators=200, max_depth=6, learning_rate=0.05, scale_pos_weight=10)
lgbm_model.fit(X_train_balanced, y_train_balanced)


[LightGBM] [Info] Number of positive: 67192, number of negative: 67192
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9448
[LightGBM] [Info] Number of data points in the train set: 134384, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(learning_rate=0.05, max_depth=6, n_estimators=200,
               random_state=42, scale_pos_weight=10)

**STEP 4:**

This code combines the previously trained models (Logistic Regression, Random Forest, XGBoost, and LightGBM) using a VotingClassifier with soft voting. The ensemble model is then trained on the balanced dataset to make final predictions based on the combined output of all individual models.

In [51]:
# Combine Models with VotingClassifier
voting_model = VotingClassifier(
    estimators=[
        ('lr', logistic_model),
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model)
    ],
    voting='soft'
)
voting_model.fit(X_train_balanced, y_train_balanced)


[LightGBM] [Info] Number of positive: 67192, number of negative: 67192
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047647 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9448
[LightGBM] [Info] Number of data points in the train set: 134384, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


VotingClassifier(estimators=[('lr',
                              LogisticRegression(class_weight='balanced',
                                                 max_iter=1000,
                                                 random_state=42)),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=12,
                                                     n_estimators=200,
                                                     random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.05, max_depth=6,
                                             n_estimators=200, random_state=42,
                                             scale_pos_weight=10))],
                 voting='soft')

In [52]:
# Evaluate Individual Models
models = {'Logistic Regression': logistic_model, 'Random Forest': rf_model, 'XGBoost': xgb_model, 'LightGBM': lgbm_model, 'Voting': voting_model}

for model_name, model in models.items():
    y_pred = model.predict(X_val_transformed)
    y_pred_proba = model.predict_proba(X_val_transformed)[:, 1]

    accuracy = accuracy_score(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    conf_matrix = confusion_matrix(y_val, y_pred)
    class_report = classification_report(y_val, y_pred)

    print(f"\n================== {model_name} Evaluation ==================")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"ROC AUC Score: {roc_auc:.2f}\n")
    print("================== Confusion Matrix ==================")
    print(pd.DataFrame(conf_matrix,
                       index=['Actual Non-Default (0)', 'Actual Default (1)'],
                       columns=['Predicted Non-Default (0)', 'Predicted Default (1)']))
    print("\n================ Classification Report ================")
    print(class_report)



================== Logistic Regression Evaluation ==================
Accuracy: 66.02%
ROC AUC Score: 0.71

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actual Non-Default (0)                       3798                   2119
Actual Default (1)                           5609                  11215

================ Classification Report ================
              precision    recall  f1-score   support

           0       0.40      0.64      0.50      5917
           1       0.84      0.67      0.74     16824

    accuracy                           0.66     22741
   macro avg       0.62      0.65      0.62     22741
weighted avg       0.73      0.66      0.68     22741


================== Random Forest Evaluation ==================
Accuracy: 65.71%
ROC AUC Score: 0.70

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actu

**Approach Summary:**

In this approach, we integrated four individual models **(Logistic Regression, Random Forest, XGBoost, and LightGBM)** using an ensemble method called Voting Classifier. The Voting Classifier combines the predictions of these models to make the final decision based on a soft voting mechanism, where each model's predicted probabilities are considered.

Here is a summary of the model performances:

**Logistic Regression: Accuracy: 66.02%, ROC AUC: 0.71**

(A decent performer but not ideal for handling imbalanced data. Struggles with predicting non-default loans.)

**Random Forest: Accuracy: 65.71%, ROC AUC: 0.70**

(Similar performance to Logistic Regression, slightly lower accuracy and ROC AUC score.)

**XGBoost: Accuracy: 73.99%, ROC AUC: 0.70**

(Shows the best accuracy but struggles with predicting non-default loans, which affects the overall performance.)

**LightGBM: Accuracy: 73.98%, ROC AUC: 0.71**

(Very similar to XGBoost, with strong recall for default loans, but low precision for non-default loans.)

**Voting Classifier: Accuracy: 74.64%, ROC AUC: 0.71**

(The ensemble model outperforms the individual models, achieving the highest accuracy and providing a more balanced prediction between default and non-default loans.)

**Conclusion:**

 Although the Voting Classifier improves the accuracy compared to individual models, it still shows room for improvement, especially in terms of precision for non-default loans. This suggests that further tuning or more advanced models may be required for better performance.



---





**Third Approach : Meta-Model**

In the third approach, we utilize a **Stacking Classifier** to combine multiple base learners.

Stacking is an ensemble learning technique that combines different models to improve overall performance. The base learners make predictions, and a meta-model is trained to predict the final outcome based on these predictions.

The base learners in this approach include:

**Random Forest (RF)**

**Gradient Boosting (GB)**

**XGBoost (XGB)**

**LightGBM (LGBM)**

The predictions from these base models are fed into a **Logistic Regression meta-model**, which learns to combine them optimally for the final prediction.

**Key Benefits:**

*   **Diversity of Models**: Using a combination of different models like Random Forest, XGBoost, and LightGBM allows the stacking model to leverage the strengths of each individual model.
*   **Meta-model for Optimal Combination:** The Logistic Regression meta-model fine-tunes the prediction by learning the best way to combine the outputs of the base models, potentially leading to better performance.





**Step 1:**

This code performs data preprocessing by loading the dataset, handling missing values, creating new features, and applying transformations to numerical and categorical columns. It also handles class imbalance using SMOTEENN and splits the dataset into training and validation sets for further model training.

In [55]:
# Load Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN

file_path = '/content/train_data.xlsx'
data = pd.read_excel(file_path)

# Explore Data
print("Dataset Shape:", data.shape)
print("Class Distribution:\n", data['loan_status'].value_counts())

# Convert 'term' and 'emp_length' to numeric before calculation
data['term'] = pd.to_numeric(data['term'], errors='coerce')
data['emp_length'] = pd.to_numeric(data['emp_length'], errors='coerce')

data['debt_to_income'] = data['loan_amnt'] / (data['annual_inc'] + 1)
data['loan_to_income'] = data['loan_amnt'] / (data['annual_inc'] + 1)
data['stability_ratio'] = data['emp_length'] / (data['term'] + 1)

# Drop Low Importance Features
data_cleaned = data.drop(columns=['customer_id', 'transaction_date'])

# Preprocessing Pipeline
categorical_cols = data_cleaned.select_dtypes(include=['object']).columns
numerical_cols = ['loan_amnt', 'annual_inc', 'cibil_score', 'account_bal',
                  'debt_to_income', 'loan_to_income', 'stability_ratio']

# Add SimpleImputer to handle missing values
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # or 'median'
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols) # sparse=False for SMOTEENN
    ]
)

# Separate Features and Target
X = data_cleaned.drop(columns=['loan_status'])
y = data_cleaned['loan_status']

# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Handle Class Imbalance
smote_enn = SMOTEENN(random_state=42)
X_train_balanced, y_train_balanced = smote_enn.fit_resample(preprocessor.fit_transform(X_train), y_train)
X_val_transformed = preprocessor.transform(X_val)


Dataset Shape: (113705, 17)
Class Distribution:
 loan_status
1    84016
0    29689
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['stability_ratio']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['stability_ratio']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


**Step 2:**

This code implements a stacking ensemble model by combining four base learners (RandomForest, GradientBoosting, XGBoost, and LightGBM) and using Logistic Regression as the meta-model to make the final prediction. The model is trained on the balanced training data using the fit method.

In [56]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Base Learners
base_learners = [
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=10, class_weight='balanced', random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6, scale_pos_weight=10, random_state=42)),
    ('lgbm', LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=6, scale_pos_weight=10, random_state=42))
]

# Meta-Model (Logistic Regression)
stacking_model = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression(max_iter=1000))

# Train Stacking Model
stacking_model.fit(X_train_balanced, y_train_balanced)


[LightGBM] [Info] Number of positive: 28965, number of negative: 42763
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8957
[LightGBM] [Info] Number of data points in the train set: 71728, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403817 -> initscore=-0.389585
[LightGBM] [Info] Start training from score -0.389585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(class_weight='balanced',
                                                       max_depth=10,
                                                       n_estimators=200,
                                                       random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(max_depth=6,
                                                           n_estimators=200,
                                                           random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device...
                                              max_delta_step=None, max_depth=6,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=200, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('lgbm',
                                LGBMClassifier(learning_rate=0.05, max_depth=6,
                                               n_estimators=200,
                                               random_state=42,
                                               scale_pos_weight=10))],
                   final_estimator=LogisticRegression(max_iter=1000))

**Step 3:**

In [57]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
import numpy as np

# Evaluate Models
y_pred_stack = stacking_model.predict(X_val_transformed)
y_pred_proba_stack = stacking_model.predict_proba(X_val_transformed)[:, 1]

accuracy_stack = accuracy_score(y_val, y_pred_stack)
roc_auc_stack = roc_auc_score(y_val, y_pred_proba_stack)
conf_matrix_stack = confusion_matrix(y_val, y_pred_stack)
class_report_stack = classification_report(y_val, y_pred_stack)

print("\n================== Stacking Model Evaluation ==================")
print(f"Accuracy: {accuracy_stack * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc_stack:.2f}\n")

print("================== Confusion Matrix ==================")
print(pd.DataFrame(conf_matrix_stack,
                   index=['Actual Non-Default (0)', 'Actual Default (1)'],
                   columns=['Predicted Non-Default (0)', 'Predicted Default (1)']))

print("\n================ Classification Report ================")
print(class_report_stack)



================== Stacking Model Evaluation ==================
Accuracy: 67.69%
ROC AUC Score: 0.71

================== Confusion Matrix ==================
                        Predicted Non-Default (0)  Predicted Default (1)
Actual Non-Default (0)                       3699                   2239
Actual Default (1)                           5109                  11694

================ Classification Report ================
              precision    recall  f1-score   support

           0       0.42      0.62      0.50      5938
           1       0.84      0.70      0.76     16803

    accuracy                           0.68     22741
   macro avg       0.63      0.66      0.63     22741
weighted avg       0.73      0.68      0.69     22741



**Stacking Approach Overview and Evaluation**

In this approach, we used a stacking ensemble model to combine the strengths of multiple base learners. The base models included:

1.   Random Forest: A versatile and powerful ensemble method using decision trees.
2.   Gradient Boosting: A boosting method that builds models sequentially to reduce errors.

3.   XGBoost: A highly efficient gradient boosting algorithm known for its speed and performance.
4.   LightGBM: A fast and efficient gradient boosting framework optimized for large datasets.



The outputs of these base models were then combined using **Logistic Regression** as the meta-model, which learned to make the final prediction based on the predictions of the base models.

**Results:**

1.   Accuracy: 67.69%

2.  ROC AUC Score: 0.71

**Conclusion:**
The stacking model combines multiple strong learners, improving model performance compared to individual models. While it achieved an accuracy of 67.69% and an ROC AUC of 0.71, it still didn't provide the optimal model performance. However, stacking generally improves predictive power, especially for imbalanced datasets, and is often a good strategy when diverse base models complement each other.





---



**Final Overview: Model Comparison and Winner Announcement**

In this project, we explored three distinct approaches to build a predictive model for loan default prediction. Each approach aimed to tackle the class imbalance and improve model performance, ultimately leading to a comprehensive evaluation of different ensemble and individual learning strategies.

1. First Approach: Individual Models
We started by evaluating Logistic Regression, Random Forest, XGBoost, and LightGBM individually. Each model was trained and evaluated on the balanced data using various metrics such as accuracy, ROC AUC score, confusion matrix, and classification report. Despite each model providing useful insights, none of the individual models achieved outstanding performance on the task.


  *   Top Performance:XGBoost and LightGBM were the best performers in this approach, achieving an accuracy of ~74%, but still faced challenges with precision and recall for the minority class.


2. Second Approach: Voting Ensemble
Next, we combined the base models using a VotingClassifier, which used a soft voting strategy to combine the predictions of Logistic Regression, Random Forest, XGBoost, and LightGBM. The voting model benefited from the strengths of each individual model, providing a better overall performance.



*   Top Performance:
Voting Model achieved an accuracy of 74.64%, with a balanced ROC AUC score of 0.71. However, it still struggled with the precision-recall trade-off, especially for the minority class (default).


3. Third Approach: Stacking Ensemble
In the final approach, we employed Stacking, combining the predictions of Random Forest, Gradient Boosting, XGBoost, and LightGBM using Logistic Regression as the meta-model. Stacking aims to utilize the diversity of base models by learning from their outputs and making more refined predictions.


*   Top Performance:
The Stacking Model achieved an accuracy of 67.69%, which was slightly lower than the voting model but still showed an improvement in prediction power compared to individual models.


**Final Outcome and Winner:**

After evaluating all three approaches, the **Voting Ensemble** emerged as the most robust model, with the highest accuracy (74.64%) and a balanced ROC AUC score (0.71). Despite some challenges with precision and recall for the minority class, the Voting model demonstrated the power of combining different algorithms for enhanced generalization.

While each approach brought valuable insights, the Voting Ensemble was the final winner in this experiment, proving that ensemble methods can significantly improve predictive performance, especially in handling imbalanced datasets like loan default prediction.

**Conclusion:**
The **Voting Ensemble model** is the best-performing model from the three approaches, making it the ideal choice for this problem. However, further tuning and advanced techniques like hyperparameter optimization or advanced ensemble methods could potentially lead to even better results.